In [1]:
import pandas as pd
import numpy as np

In [2]:
trees = pd.read_csv("ESP_PUBLIC.IDENTITE_ARBRE.csv", header="infer")

In [3]:
trees.head()

,ELEM_POINT_ID,CODE,NOM,GENRE,GENRE_DESC,CATEGORIE,CATEGORIE_DESC,SOUS_CATEGORIE,SOUS_CATEGORIE_DESC,CODE_PARENT,...,COURRIER,IDENTIFIANTPLU,TYPEIMPLANTATIONPLU,INTITULEPROTECTIONPLU,ANNEEABATTAGE,ESSOUCHEMENT,DIAMETREARBRE,CAUSEABATTAGE,COLLECTIVITE,GeoJSON
0,16750,ESP19318,ESP19318,VEG,VEGETATION,ESP01,Arbre,ESP174,Arbre d'espaces ouverts,ESP995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Grenoble,"{""type"":""Point"",""coordinates"":[5.7394161255246..."
1,20142,ESP18095,ESP18095,VEG,VEGETATION,ESP01,Arbre,ESP174,Arbre d'espaces ouverts,ESP995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Grenoble,"{""type"":""Point"",""coordinates"":[5.7397428849165..."
2,23843,ESP17861,ESP17861,VEG,VEGETATION,ESP01,Arbre,ESP174,Arbre d'espaces ouverts,ESP995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Grenoble,"{""type"":""Point"",""coordinates"":[5.7393229609447..."
3,23841,ESP17860,ESP17860,VEG,VEGETATION,ESP01,Arbre,ESP174,Arbre d'espaces ouverts,ESP995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Grenoble,"{""type"":""Point"",""coordinates"":[5.7410808202830..."
4,1778,ESP16403,ESP16403,VEG,VEGETATION,ESP01,Arbre,ESP174,Arbre d'espaces ouverts,ESP995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Grenoble,"{""type"":""Point"",""coordinates"":[5.7408124079286..."


In [4]:
trees[["GeoJSON"]].iloc[1,:].values[0]

'{"type":"Point","coordinates":[5.73974288491655,45.1908047491604]}'

In [5]:
import folium
from folium import plugins

In [6]:
m_1 = folium.Map(location=[45.1885, 5.7245], zoom_start=(15))

folium.Marker([45.1911217700938, 5.73941612552464]).add_to(m_1)
folium.Marker([45.1908047491604, 5.73974288491655]).add_to(m_1)

m_1

In [7]:
geoData = trees["GeoJSON"] != ""
location = trees[geoData]["GeoJSON"].values

In [8]:
# list of strings GeoJSON
data_location = [x for x in location]
type(data_location[0])

str

In [9]:
# write column in a file as json.....
trees[["GeoJSON"]].to_string("geodata.json", header=False, index=False)

In [ ]:
# What type you want to send ?
# How many items you plan send 
# Attention : Is it a good idea to show 30k markers ?

In [ ]:
m = folium.Map(location=[45.1885, 5.7245], zoom_start=(15))
location = trees[["GeoJSON"]].iloc[:,:].values
folium.GeoJson(
    data_location[0],
    name='arbres'
).add_to(m)

In [ ]:
# it kills the RAM!!!!!
#----------------------------------------------------------
m = folium.Map(location=[45.1885, 5.7245], zoom_start=(15))

with open("geodata.json", "r") as geodata:
    for line in geodata:
        folium.GeoJson(line, name='arbres').add_to(m)


In [10]:
data_geo = pd.read_json("geodata.json", lines=True)

In [11]:
data_geo.head()

,type,coordinates
0,Point,"[5.73941612552464, 45.1911217700938]"
1,Point,"[5.73974288491655, 45.1908047491604]"
2,Point,"[5.73932296094475, 45.1910917297403]"
3,Point,"[5.74108082028307, 45.1892629840984]"
4,Point,"[5.7408124079286, 45.1903250401832]"


In [12]:
data_geo["coordinates"] = [x[::-1] for x in data_geo["coordinates"]]

In [13]:
type(data_geo[["coordinates"]].values[0][0])

list

In [14]:
data_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31892 entries, 0 to 31891
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   type         31892 non-null  object
 1   coordinates  31892 non-null  object
dtypes: object(2)
memory usage: 498.4+ KB


In [15]:
data_geo.head()

,type,coordinates
0,Point,"[45.1911217700938, 5.73941612552464]"
1,Point,"[45.1908047491604, 5.73974288491655]"
2,Point,"[45.1910917297403, 5.73932296094475]"
3,Point,"[45.1892629840984, 5.74108082028307]"
4,Point,"[45.1903250401832, 5.7408124079286]"


In [16]:
data_2 = np.array([x for x in data_geo["coordinates"]])

In [17]:
data_2

array([[45.19112177,  5.73941613],
       [45.19080475,  5.73974288],
       [45.19109173,  5.73932296],
       ...,
       [45.16952655,  5.72532546],
       [45.1704092 ,  5.72620504],
       [45.17085716,  5.72513683]])

In [18]:
m = folium.Map(location=[45.1885, 5.7245], zoom_start=(15))
plugins.FastMarkerCluster(data_2).add_to(m)

In [19]:
m

### Calculating the distance between trees

In [20]:
data_geo["coordinates"][0]

[45.1911217700938, 5.73941612552464]

In [21]:
cord1 = data_geo["coordinates"][0]
cord2 = data_geo["coordinates"][1]

#### Using HARVESINE formula

In [22]:
from math import radians, cos, sin, asin, sqrt

# Harvesine formula:
# a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)
# c = 2 ⋅ atan2( √a, √(1−a) )
# d = R ⋅ c


def harvesine(cord1, cord2):
    lon1, lat1, lon2, lat2 = map(radians, [cord1[0], cord1[1], cord2[0], cord2[1]])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [23]:
harvesine(cord1, cord2)

0.05050120617394483

In [24]:
# my coordinates: [45.19835182199177, 5.714207998083206]
chez_moi = [45.19835182199177, 5.714207998083206]

In [25]:
type(chez_moi)

list

In [26]:
data_geo["dist_chezMoi"] = [harvesine(chez_moi, x) for x in data_geo["coordinates"]]

In [27]:
data_geo.head()

,type,coordinates,dist_chezMoi
0,Point,"[45.1911217700938, 5.73941612552464]",2.914925
1,Point,"[45.1908047491604, 5.73974288491655]",2.959585
2,Point,"[45.1910917297403, 5.73932296094475]",2.905882
3,Point,"[45.1892629840984, 5.74108082028307]",3.152789
4,Point,"[45.1903250401832, 5.7408124079286]",3.088702


In [28]:
proche = data_geo.sort_values("dist_chezMoi", ascending=True, inplace=False).head(10)
data_geo.sort_values("dist_chezMoi", ascending=True, inplace=False).head(10)

,type,coordinates,dist_chezMoi
6664,Point,"[45.1983704763642, 5.71418753973526]",0.003072
6665,Point,"[45.1984287275833, 5.7141759144959]",0.009227
6582,Point,"[45.1984719326072, 5.71424126147767]",0.013794
6581,Point,"[45.1984975022952, 5.71423528033473]",0.016401
10780,Point,"[45.1984173628714, 5.71441573707811]",0.024211
10802,Point,"[45.1983288160198, 5.71442600816222]",0.024375
6583,Point,"[45.198117555044, 5.71425899143504]",0.026533
10892,Point,"[45.1985059023389, 5.7144057202642795]",0.027821
10792,Point,"[45.1982380622897, 5.7144347501005]",0.028181
6584,Point,"[45.1980890736039, 5.714259080082]",0.029621


In [29]:
m_proche = folium.Map(location=[45.1885, 5.7245], zoom_start=(15))

for item in [x for x in proche["coordinates"]]:
    folium.Marker(item).add_to(m_proche)

#### the 10 closest trees to my place...........

In [30]:
m_proche

### Using geopy formula

In [32]:
from geopy.distance import geodesic, great_circle

def geopy_dist(cord1, cord2):
    return geodesic(cord1, cord2).kilometers

In [33]:
harvesine(cord1, cord2)

0.05050120617394483

In [34]:
geopy_dist(cord1, cord2)

0.04359684251266069

In [35]:
data_geo["dist_chezMoi_geopy"] = [geopy_dist(chez_moi, x) for x in data_geo["coordinates"]]

In [36]:
data_geo.head()

,type,coordinates,dist_chezMoi,dist_chezMoi_geopy
0,Point,"[45.1911217700938, 5.73941612552464]",2.914925,2.137604
1,Point,"[45.1908047491604, 5.73974288491655]",2.959585,2.174768
2,Point,"[45.1910917297403, 5.73932296094475]",2.905882,2.132084
3,Point,"[45.1892629840984, 5.74108082028307]",3.152789,2.340831
4,Point,"[45.1903250401832, 5.7408124079286]",3.088702,2.272940


In [37]:
proche_geopy = data_geo.sort_values("dist_chezMoi", ascending=True, inplace=False).head(10)
data_geo.sort_values("dist_chezMoi", ascending=True, inplace=False).head(10)

,type,coordinates,dist_chezMoi,dist_chezMoi_geopy
6664,Point,"[45.1983704763642, 5.71418753973526]",0.003072,0.002623
6665,Point,"[45.1984287275833, 5.7141759144959]",0.009227,0.008911
6582,Point,"[45.1984719326072, 5.71424126147767]",0.013794,0.013602
6581,Point,"[45.1984975022952, 5.71423528033473]",0.016401,0.016332
10780,Point,"[45.1984173628714, 5.71441573707811]",0.024211,0.017874
10802,Point,"[45.1983288160198, 5.71442600816222]",0.024375,0.017320
6583,Point,"[45.198117555044, 5.71425899143504]",0.026533,0.026342
10892,Point,"[45.1985059023389, 5.7144057202642795]",0.027821,0.023121
10792,Point,"[45.1982380622897, 5.7144347501005]",0.028181,0.021847
6584,Point,"[45.1980890736039, 5.714259080082]",0.029621,0.029475


In [38]:
m_proche_geopy = folium.Map(location=[45.1885, 5.7245], zoom_start=(15))

for item in [x for x in proche_geopy["coordinates"]]:
    folium.Marker(item).add_to(m_proche_geopy)

In [39]:
m_proche_geopy

### Distance between Trees

In [40]:
from sklearn.neighbors import DistanceMetric

dist = DistanceMetric.get_metric('haversine')

In [41]:
dist_pairs = dist.pairwise(np.array([x for x in data_geo["coordinates"]]))

In [42]:
dist_pairs.shape

(31892, 31892)

In [43]:
# example from array
dist_pairs[0][5]

5.281024145008276

In [44]:
# maximal distance between trees is 490 m????
np.max(dist_pairs), np.mean(dist_pairs)

(489.56449967172017, 99.73601332135598)

In [46]:
m_longest = folium.Map(location=[45.1885, 5.7245], zoom_start=(15))

for i in range(len(dist_pairs)):
    for j in range(len(dist_pairs)):
        if dist_pairs[i, j] == 489.56449967172017:
            print("index: [{}, {}]".format(i, j))
            print("Distance harvesine: {}".format(harvesine(data_geo["coordinates"][i], data_geo["coordinates"][j])))                           
            print("Distance geopy: {}".format(geopy_dist(data_geo["coordinates"][i], data_geo["coordinates"][j])))
            folium.Marker(data_geo["coordinates"][i]).add_to(m_longest)
            folium.Marker(data_geo["coordinates"][j]).add_to(m_longest)
            
m_longest             

index: [5953, 8620]
Distance harvesine: 10.886517234778337
Distance geopy: 9.626509086133748
index: [8620, 5953]
Distance harvesine: 10.886517234778337
Distance geopy: 9.626509086133748


In [ ]:
from scipy.spatial.distance import cdist

In [ ]:
# using scipy.....
dist_pair_2 = cdist(np.array([x for x in data_geo["coordinates"]]),
                   np.array([x for x in data_geo["coordinates"]]),
                   harvesine)

In [76]:
l = []
for i in range(1, len(trees) + 2):
    if (len(trees)+1) % i == 0:
        l.append(i)

In [77]:
l

[1, 3, 10631, 31893]

In [68]:
31892 / 268

119.0

In [ ]:
119 * 

In [57]:
from tqdm import tqdm

dist_pair_3 = np.zeros((len(trees), len(trees)))

i = 0
k = 268
while i*k < len(trees):
    for ind in tqdm(range((i * k), (i + 1) * k - 1)):
        for jnd in range(len(trees)):
            dist_pair_3[ind,jnd] = harvesine(data_geo["coordinates"][ind], data_geo["coordinates"][jnd])
    i += 1
    if i == 119:
        for x in range(len(trees)):
            dist_pair_3[len(trees),x] = harvesine(data_geo["coordinates"][len(trees)], data_geo["coordinates"][x])
            
        
    

 56%|██████████████████████▍                 | 150/267 [01:13<00:57,  2.04it/s]


KeyboardInterrupt: 

In [52]:
i

119

In [53]:
dist_pair_3[0,0]

0.0

In [54]:
np.max(dist_pair_3)

12.45666297999058

In [56]:
m_longest2 = folium.Map(location=[45.1885, 5.7245], zoom_start=(15))

for i in range(len(dist_pair_3)):
    for j in range(len(dist_pair_3)):
        if dist_pair_3[i, j] == 12.45666297999058:
            print("index: [{}, {}]".format(i, j))
            print("Distance harvesine: {}".format(harvesine(data_geo["coordinates"][i], data_geo["coordinates"][j])))                           
            print("Distance geopy: {}".format(geopy_dist(data_geo["coordinates"][i], data_geo["coordinates"][j])))
            folium.Marker(data_geo["coordinates"][i]).add_to(m_longest2)
            folium.Marker(data_geo["coordinates"][j]).add_to(m_longest2)
            
m_longest2           

index: [5952, 6099]
Distance harvesine: 12.45666297999058
Distance geopy: 10.086889728784737
index: [6099, 5952]
Distance harvesine: 12.45666297999058
Distance geopy: 10.086889728784737
